<!-- dom:TITLE: Exercises from Linear algebra, signal processing, and wavelets. A unified approach.\\ Python version -->
# Exercises from Linear algebra, signal processing, and wavelets. A unified approach.\\ Python version
<!-- dom:AUTHOR: Øyvind Ryan -->
<!-- Author: --> **Øyvind Ryan**

Date: **Oct 24, 2018**

<!-- Externaldocuments: applinalg -->
<!-- Mapping from exercise labels to numbers: label2numbers = {'exercise:nonrecursivealg': '2.29', 'exercise:compareextimes': '2.25', 'exercise:interpolant': '2.21', 'example:dftmainex2': '2.5', 'example:dftmainex3': '2.6'} -->

# Digital sound and Discrete Fourier analysis








<!-- --- begin exercise --- -->

## Example 2.5: Thresholding the DFT coefficients
<div id="example:dftmainex2"></div>
<!-- keywords = fouriervectors -->

In [1]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), 'python'))

In [2]:
%matplotlib inline

from numpy import *
import time
from sound import *
from forward_compress_reverse import *
import matplotlib.pyplot as plt
from scipy.fftpack import dct, idct

Since the DFT coefficients represent contributions at given frequencies, they are extremely useful for performing operations on sound, and also for compression. The function `forw_comp_rev_DFT`
in the module `forward_compress_reverse` 
in the library can manipulate the DFT coefficients of the audio sample file in several ways, and plays the result. 
In particular it can set DFT coefficients below a given threshold to zero. The idea behind doing so is that frequency components with small values
may contribute little to the perception of the sound, so that they can be discarded. This gives a sound with fewer frequencies, 
more suitable for compression. `forw_comp_rev_DFT` accepts the named parameter `threshold` for this purpose.  
This parameter will also force writing the percentage of the DFT coefficients which are zeroed out to the display.

Let us first listen to the result when `threshold=20`.

In [3]:
x, fs = forw_comp_rev_DFT(threshold=20)
play(x, fs)

Then for `threshold=70`.

In [4]:
x, fs = forw_comp_rev_DFT(threshold=70)
play(x, fs)

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 2.6: Quantizing the DFT coefficients
<div id="example:dftmainex3"></div>
<!-- keywords = fouriervectors -->

The thresholding from the previous example affects only frequencies with low contribution. 
This makes it too simple for practical use:
A more neutral way would be to let each DFT coefficient occupy a certain number of bits. This is also called *quantization*, and is closer to what modern audio standards do. 
This is useful if the number of bits we use is less than what we have at the start.
`forw_comp_rev_DFT` accepts a named parameter `n`, which secures that a DFT coefficient with bit representation

$$
...d_2d_1d_0.d_{-1}d_{-2}d_{-3}...
$$

is truncated so that the bits $d_{n-1},d_{n-2},...$ are discarded. High values of $n$ thus mean more rounding, 
so that one should hear that the sound degrades with increasing $n$. The following three examples verify this:
Let us first listen to the result for `n=3`.

In [5]:
x, fs = forw_comp_rev_DFT(n=3)
play(x, fs)

Then for `n=5`.

In [6]:
x, fs = forw_comp_rev_DFT(n=5)
play(x, fs)

Then for `n=7`.

In [7]:
x, fs = forw_comp_rev_DFT(n=7)
play(x, fs)

In practice this quantization procedure is also too simple, since the
human auditory system is more sensitive to certain frequencies, and should thus allocate a higher number of bits for those. 
Modern audio standards take this into account.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 2.21: Plotting the interpolant
<div id="exercise:interpolant"></div>
<!-- keywords = fouriervectors; student -->

Implement code where you do the following:

* at the top you define the function $f(x)=\cos^6(x)$, and $M=3$,

* compute the unique interpolant from $V_{M,T}$ (i.e. by taking $N=2M+1$ samples over one period), 
  as guaranteed by Proposition 9,

* plot the interpolant against $f$ over one period.

Finally run the code also for $M=4$, $M=5$, and $M=6$. Explain why the plots coincide for $M=6$, but not for $M < 6$. Does increasing $M$ above $M=6$ have any effect on the plots?


<!-- --- begin solution of exercise --- -->
**Solution.**
The code can look as follows.

In [8]:
import matplotlib.pyplot as plt
from numpy import *

f = lambda t:cos(t)**6
M = 5
T = 2*pi
N = 2*M + 1
t = linspace(0, T, 100)
x = f(linspace(0, T - T/float(N), N))
y = fft.fft(x, axis=0)/N
s = real(y[0])*ones(len(t))
for k in range(1,int((N+1)/2)): 
    s += 2*real(y[k]*exp(2*pi*1j*k*t/float(T)))
plt.plot(t, s, 'r', t, f(t), 'g')
plt.legend(['Interpolant from V_{M,T}','f'])

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 2.25: Execution times for the FFT
<div id="exercise:compareextimes"></div>
<!-- keywords = fouriervectors; student -->

Let us compare execution times for the different methods for computing the DFT.


**a)**
Write code which compares the execution times for an $N$-point DFT for the following three cases: Direct implementation of the DFT 
(as in Example 2.4),
the FFT implementation used in this chapter, and the built-in `fft`-function. Your code should use the sample audio file `castanets.wav`, 
apply the different DFT implementations to the first $N=2^r$ samples of the file for $r=3$ to $r=15$, store the execution times in a vector, and plot these. You can use 
the function `time()` in the `time` module
to measure the execution time.


<!-- --- begin solution of exercise --- -->
**Solution.**
The code can look as follows.

In [9]:
x0, fs = audioread('sounds/castanets.wav')

kvals = arange(3,16)
slowtime = zeros(len(kvals))
fasttime = zeros(len(kvals))
fastesttime = zeros(len(kvals))
N = 2**kvals
for k in kvals:
    x = x0[0:2**k]
    x = x.astype(complex)
    
    start = time.time()
    dft_impl(x)
    slowtime[k - kvals[0]] = time.time() - start
    
    start = time.time()
    fft_impl(x, fft_kernel_standard)
    fasttime[k - kvals[0]] = time.time() - start

    start = time.time()
    fft.fft(x, axis=0)
    fastesttime[k - kvals[0]] = time.time() - start

# a.
plt.plot(kvals, slowtime, 'ro-', \
     kvals, fasttime, 'go-', \
     kvals, fastesttime, 'bo-')
plt.grid('on')
plt.title('time usage of the DFT methods')
plt.legend(['DFT', 'Standard FFT', 'Built-in FFT'])
plt.xlabel('log_2 N')
plt.ylabel('time used [s]')
plt.show()
 
plt.figure()

# b.
plt.loglog(N, slowtime, 'ro-', N, fasttime, 'go-', N, fastesttime, 'bo-')
plt.axis('equal')
plt.legend(['DFT', 'Standard FFT', 'Built-in FFT'])

<!-- --- end solution of exercise --- -->


**b)**
A problem for large $N$ is that there is such a big difference in the execution times between the two implementations. We can address this by using a loglog-plot. Plot $N$ against execution
times using the function `loglog`. How should the fact that the number of arithmetic operations are $8N^2$ and $5N\log_2N$ be reflected in the plot?


<!-- --- begin solution of exercise --- -->
**Solution.**
The two different curves you see should have a derivative approximately equal to one and two, respectively.

<!-- --- end solution of exercise --- -->

**c)**
It seems that the built-in FFT is much faster than our own FFT implementation, even though they may use similar algorithms. Try to explain what can be the cause of this.


<!-- --- begin solution of exercise --- -->
**Solution.**
There may be several reasons for this. 
One is that Python code runs slowly when compared to native code, which is much used in the built-in FFT. Also, the built-in `fft` has been subject to much more optimization than we have
covered here.

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 2.29: Non-recursive FFT algorithm
<div id="exercise:nonrecursivealg"></div>
<!-- keywords = fouriervectors; student -->

Use the factorization in (2.16) to write a kernel function for a non-recursive FFT implementation.
In your code, perform the matrix multiplications in equation (2.16) 
from right to left in an (outer) for-loop. For each matrix loop through the different blocks on the diagonal in
an (inner) for-loop. Make sure you have the right number of blocks on the diagonal, each block being on the form

$$
\begin{pmatrix} I & D_{N/2^k} \\ I & -D_{N/2^k} \end{pmatrix}.
$$

It may be a good idea to start by implementing multiplication with such a simple matrix first as these are the building blocks in the algorithm. 
Do this so that everything is computed in-place. Also compare the execution times with our original FFT algorithm, as we did in 
Exercise 2.25, and try to explain what you see in this comparison.


<!-- --- begin solution of exercise --- -->
**Solution.**
The library version of the non-recursive FFT is as follows

        def fft_kernel_nonrec(x, forward):
            """
            Compute the FFT of x, using a non-recursive FFT algorithm. 
            
            x: a bit-reversed version of the input. Should have only one axis
            forward: Whether the FFT or the IFFT is applied
            """
            N = len(x)
            sign = -1
            if not forward:
                sign = 1
            D = exp(sign*2*pi*1j*arange(float(N/2))/N)
            nextN = 1
            while nextN < N:
                k = 0
                while k < N:
                    xe, xo = x[k:(k + nextN)], x[(k + nextN):(k + 2*nextN)]
                    xo *= D[0::(int(N/(2*nextN)))]
                    x[k:(k+2*nextN)] = concatenate([xe + xo, xe - xo])
                    k += 2*nextN
                nextN *= 2


If you add the non-recursive algorithm to the code from Exercise 2.25, you will see that the non-recursive algorithm performs much better. There may be several
reasons for this. First of all, there are no recursive function calls. Secondly, the values in the matrices $D_{N/2}$ are constructed once and for all with
the non-recursive algorithm. Code which compares execution times for the original FFT algorithm, our non-recursive implementation, and the split-radix algorithm of the next
exercise, can look as follows:

In [10]:
x0, fs = audioread('sounds/castanets.wav') 

kvals = arange(3,16)
slowtime = zeros(len(kvals))
fasttime = zeros(len(kvals))
fastesttime = zeros(len(kvals))
N = 2**kvals
for k in kvals:
    x = x0[0:2**k]
    x = x.astype(complex)
    
    start = time.time()
    fft_impl(x, fft_kernel_standard)
    slowtime[k - kvals[0]] = time.time() - start
    
    start = time.time()
    fft_impl(x, fft_kernel_nonrec)
    fasttime[k - kvals[0]] = time.time() - start

    start = time.time()
    fft_impl(x, fft_kernel_splitradix)
    fastesttime[k - kvals[0]] = time.time() - start

plt.plot(kvals, slowtime, 'ro-', \
     kvals, fasttime, 'bo-', \
     kvals, fastesttime, 'go-')
plt.grid('on')
plt.title('time usage of the DFT methods')
plt.legend(['Standard FFT algorithm', \
       'Non-recursive FFT', \
       'Split radix FFT'])
plt.xlabel('log_2 N')
plt.ylabel('time used [s]')

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->